**Another question: what happens when we make sure that there is at least 30 mins between each class?**

Pull CSV I scraped from the UNC-CH course registrar pdf.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

fall19_courses = pd.read_csv("UNCfall19courses.csv")

df = fall19_courses["Time:"].str.split(r'[:,-]', expand=True)
df_start = (df[0] + df[1])
df_end =  (df[2] + df[3])


fall19_courses["start_time"] = df_start
fall19_courses["end_time"] = df_end

fall19_courses = fall19_courses.dropna(subset=['start_time'])
fall19_courses = fall19_courses.dropna(subset=['end_time'])

# working with datetime
fall19_courses["start_time"] = pd.to_datetime(fall19_courses["start_time"].astype(int), format="%H%M")
fall19_courses["end_time"] = pd.to_datetime(fall19_courses["end_time"].astype(int), format="%H%M")

In [2]:
fall19_courses = fall19_courses[fall19_courses['Bldg:']!='TBA']
fall19_courses = fall19_courses[fall19_courses['Days:']!='TBA']
# print(fall19_courses.count())
fall19_courses.head()

Bldg: Room: Days:          Time:  Class Enrl Tot:  \
AAAD  130        Gardner  0308  TuTh  12:30 - 13:45               43   
      130       Phillips  0247  TuTh  09:30 - 10:45               42   
      130        Peabody  0311  TuTh  14:00 - 15:15               43   
      200        Peabody  0306  TuTh  14:00 - 15:15               36   
      201  Carolina Hall  0220   MWF  08:00 - 08:50               32   

                   start_time            end_time  
AAAD  130 1900-01-01 12:30:00 1900-01-01 13:45:00  
      130 1900-01-01 09:30:00 1900-01-01 10:45:00  
      130 1900-01-01 14:00:00 1900-01-01 15:15:00  
      200 1900-01-01 14:00:00 1900-01-01 15:15:00  
      201 1900-01-01 08:00:00 1900-01-01 08:50:00

**Optional: remove classes with over 50 or over 100 students.**

In [3]:
fall19_courses = fall19_courses[fall19_courses['Class Enrl Tot:'] < 100]
fall19_courses = fall19_courses[fall19_courses['Class Enrl Tot:'] < 50]
print(f'Courses with more than 100 students: {fall19_courses[fall19_courses["Class Enrl Tot:"] > 99].count()[0]}')
print(f'Courses with more than 50 students: {fall19_courses[fall19_courses["Class Enrl Tot:"] > 49].count()[0]}')


Courses with more than 100 students: 0
Courses with more than 50 students: 0


**Organize by day of week into separate day names**

In [4]:

monday_courses = fall19_courses[(fall19_courses['Days:'] == 'M') 
                        | (fall19_courses['Days:'] == 'MWF') 
                        | (fall19_courses['Days:'] == 'MW')
                        | (fall19_courses['Days:'] == 'MTWTh')
                        | (fall19_courses['Days:'] == 'MTuWThFSaS')
                          ].groupby(['Bldg:','Room:','Days:','start_time', 'end_time']
                                   ).sum().reset_index().sort_values(by=['Bldg:', 'Room:', 'start_time'])

tuesday_courses = fall19_courses[(fall19_courses['Days:'] == 'Tu') 
                         | (fall19_courses['Days:'] == 'TuTh')
                         | (fall19_courses['Days:'] == 'MTWTh')
                         | (fall19_courses['Days:'] == 'MTuWThFSaS')
                           ].groupby(['Bldg:','Room:','Days:','start_time', 'end_time']
                                   ).sum().reset_index().sort_values(by=['Bldg:', 'Room:', 'start_time'])

wednesday_courses = fall19_courses[(fall19_courses['Days:'] == 'W') 
                           | (fall19_courses['Days:'] == 'MWF')
                           | (fall19_courses['Days:'] == 'MW')
                           | (fall19_courses['Days:'] == 'MTWTh')
                           | (fall19_courses['Days:'] == 'MTuWThFSaS')
                             ].groupby(['Bldg:','Room:','Days:','start_time', 'end_time']
                                   ).sum().reset_index().sort_values(by=['Bldg:', 'Room:', 'start_time'])

thursday_courses = fall19_courses[(fall19_courses['Days:'] == 'TuTh') 
                          | (fall19_courses['Days:'] == 'Th')
                          | (fall19_courses['Days:'] == 'MTWTh')
                          | (fall19_courses['Days:'] == 'MTuWThFSaS')
                            ].groupby(['Bldg:','Room:','Days:','start_time', 'end_time']
                                   ).sum().reset_index().sort_values(by=['Bldg:', 'Room:', 'start_time'])

friday_courses = fall19_courses[(fall19_courses['Days:'] == 'WF') 
                        | (fall19_courses['Days:'] == 'MWF')
                        | (fall19_courses['Days:'] == 'F')
                        | (fall19_courses['Days:'] == 'MTuWThFSaS')
                          ].groupby(['Bldg:','Room:','Days:','start_time', 'end_time']
                                   ).sum().reset_index().sort_values(by=['Bldg:', 'Room:', 'start_time'])

monday_courses.head()


,Bldg:,Room:,Days:,start_time,end_time,Class Enrl Tot:
3,Alumni,0203,MWF,1900-01-01 09:05:00,1900-01-01 09:55:00,18
4,Alumni,0203,MWF,1900-01-01 10:10:00,1900-01-01 11:00:00,22
5,Alumni,0203,MWF,1900-01-01 11:15:00,1900-01-01 12:05:00,15
0,Alumni,0203,M,1900-01-01 12:20:00,1900-01-01 13:10:00,20
1,Alumni,0203,M,1900-01-01 13:25:00,1900-01-01 15:55:00,8


In [5]:
from datetime import datetime, timedelta

day_list = [monday_courses, tuesday_courses, wednesday_courses, thursday_courses, friday_courses]
i = 0
for df in day_list:
    # initialize old bldg, room, times
    old_bldg = df.iloc[0]['Bldg:'] 
    old_room = df.iloc[0]['Room:'] 
    classbefore_starttime = df.iloc[0]['start_time']
    classbefore_endtime = df.iloc[0]['end_time']
    
    # start iterating at second row
    for index, row in df.iloc[1:].iterrows():
        new_bldg = row['Bldg:']
        new_room = row['Room:']
        class_starttime = row['start_time']
        class_endtime = row['end_time']
        if new_room == old_room and new_bldg == old_bldg:
            # find time diff between classes (can be negative!)
            diff = (class_starttime-classbefore_endtime).total_seconds()/60
            if diff < 30:
                # amount of time to add
                time_add = timedelta(minutes = int(30 - int(diff)))
                # add time
                df.at[index,'start_time'] =  row['start_time'] + time_add
                df.at[index,'end_time']   =  row['end_time'] + time_add

                classbefore_starttime = class_starttime + time_add
                classbefore_endtime = class_endtime + time_add

            else:
                classbefore_starttime = class_starttime 
                classbefore_endtime = class_endtime
                
        # in a new room or building
        else:
            classbefore_starttime = class_starttime 
            classbefore_endtime = class_endtime
                
        old_bldg = new_bldg
        old_room = new_room

In [6]:
monday_courses.head()

,Bldg:,Room:,Days:,start_time,end_time,Class Enrl Tot:
3,Alumni,0203,MWF,1900-01-01 09:05:00,1900-01-01 09:55:00,18
4,Alumni,0203,MWF,1900-01-01 10:25:00,1900-01-01 11:15:00,22
5,Alumni,0203,MWF,1900-01-01 11:45:00,1900-01-01 12:35:00,15
0,Alumni,0203,M,1900-01-01 13:05:00,1900-01-01 13:55:00,20
1,Alumni,0203,M,1900-01-01 14:25:00,1900-01-01 16:55:00,8


Save as CSV file

In [7]:
days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday']
for index, course in enumerate(day_list):
    course.to_csv(f'{days[index]}_no50.csv')